# Artificial Neural Networks: Multilayer Perceptrons

We have seen that the single Perceptron is able to linearly separate a dataset, spitting out "0" or "1" as a function of the data being below or above the separating hyperplane defined by the weight vector $w$. It is easy to see that some problems cannot be linearly separated. 

<center>
<img src="figs/xorproblem.svg">
</center>

In the example above, the red and the blue data points are not separable by a single line, but require at least two. This problem is known as the "XOR" problem, which can be seen by looking at just four data points at $(0,0)$, $(0,1)$, $(1,0)$, and $(1,1)$. Creating a truth table (right) reveals a truth table with the characteristics of logical exclusive or (XOR), that is $x_1$ and $x_2$ have to be different for the output to be true (here "blue"), whereas the output is false (here "red") when the inputs are the same.

We already know that a single Perceptron can create a single separating hyperplane, we will therefore need at least two Perceptrons to solve the XOR problem. Using two Perceptrons in parallel will yield us with tuples of the kind $(0,0)$, $(0,1)$ and so on. We therefore need one more Perceptron to recombine these tuples into a single output. The figure below shows the simple-most multi-layer Perceptron with one input layer, a so-called hidden layer, and an output layer.

<center>
<img src="figs/basicnetworks.svg">
</center>    
    

Note that each Perceptron comes with its own weight vector $w$. Given the input being of dimension $m$, the multi-layer perceptron with two Perceptrons in the hidden layer therefore has $2m+2+3$ parameters, which account for the $m$-dimensional input to the two Perceptrons in the hidden layer, the 2 parameters of the output layer Perceptron, and the three bias values. 

The output is computed by first computing the output from the two Perceptrons in the hidden layer and stacking their outputs into a vector, which serves as input vector to the Output perceptron. This is called <i>forward propagation</i>.

## Activation functions

Using a on-off Heaviside step function makes training a neural network using backpropagation rather difficult as a function that switches from "not working at all" to "working completely" provides very little information in which direction to move. It is therefore more desirable to have a smooth activation function. One such function is the <a href="https://en.wikipedia.org/wiki/Sigmoid_function">sigmoid function</a>:

$$\sigma(x)=\frac{1}{1+e^{-x}}$$

Its main characteristics are that it stays between 0 and 1, and crosses the y-axis at 0.5

<center>
<img src="figs/sigmoid.svg" width="50%">
</center>

The sigmoid function is very attractive for learning using backpropagation as the direction in which the weights should move to improve the error is very clear in the vicinity of $wx=0$. In case of $wx$ being very large, or very small, the neuron either saturates or never activates, also known as the <i>vanishing gradient</i> problem. Another drawback is that computing the sigmoid function is computationally expensive. A similar function is the hyperbolic tangent $\tanh()$ which remains in the range of -1 to 1 and crosses the y-axis at 0.

A popular solution to decrease computation time is the Rectified Linear Unit (ReLU), which is given by

$$R(x)=max(0,x)$$

and is shown in the figure below
<center>
<img src="figs/ReLU.svg" width="50%">
</center>
the dashed line indicates a refinement of the ReLU known as "leaky ReLU" with a slope of typical 0.1, and improves learning for negative $wx$ by providing a directional gradient. 


## Training a multi-layer perceptron

Finding a set of weights and bias values, that is 9 parameters for a simple two-dimensional problem, manually is an NP-complete problem (Blum, 1992). An approximative solution is known as <i>backpropagation</i> (Rumelhart, 1985). As with the single Perceptron, backpropagation computes the error between the output of the neural network for a number of known input-output pairs (the trainingset). The algorithm then tries to find small variations of the weights that will reduce the error for every trial. For this, the algorithm works backward through the network, first trying to find slightly better weights for the output layer, then for the hidden layers, and so on. Finding better weights is akin to calculating the derivative of the error with respect to the different weights, layer by layer, and moving along the gradient that will minimize the error.

This process is very slow, requiring thousands of forward and back-propagation steps for each training element. This is also the reason, neural networks have remained in obscurity for so long. Computers were simply not powerful enough to deal with millions of high-dimensional inputs - a 640x480 image has dimension $m=307,200$ - and large networks with tens of thousand of weights. It is not only computers that have become faster, but the community has also learned how to better break the learning problem apart, resulting in a number of breakthroughs in a variety of fields that deep learning has become famous for.



Code from <a href="https://brilliant.org/wiki/backpropagation/">Brilliant.org</a>:

In [27]:
import numpy as np

# define the sigmoid function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# choose a random seed for reproducible results
np.random.seed(1)

# learning rate
alpha = .1

# number of nodes in the hidden layer
num_hidden = 100

# inputs
X = np.array([  
    [0, 0],
    [0, 1],
    [1, 0],
    [1, 1],
])

# outputs
# x.T is the transpose of x, making this a column vector
y = np.array([[0, 1, 1, 0]]).T

# initialize weights randomly with mean 0 and range [-1, 1]
# the +1 in the 1st dimension of the weight matrices is for the bias weight
hidden_weights = 2*np.random.random((X.shape[1] + 1, num_hidden)) - 1
output_weights = 2*np.random.random((num_hidden + 1, y.shape[1])) - 1

# number of iterations of gradient descent
num_iterations = 1000

# for each iteration of gradient descent
for i in range(num_iterations):

    # forward phase
    # np.hstack((np.ones(...), X) adds a fixed input of 1 for the bias weight
    input_layer_outputs = np.hstack((np.ones((X.shape[0], 1)), X))
    hidden_layer_outputs = np.hstack((np.ones((X.shape[0], 1)), sigmoid(np.dot(input_layer_outputs, hidden_weights))))
    output_layer_outputs = np.dot(hidden_layer_outputs, output_weights)

    # backward phase
    # output layer error term
    output_error = output_layer_outputs - y
    # hidden layer error term
    # [:, 1:] removes the bias term from the backpropagation
    # Note that the derivative of the activation function sigma'(x)=sigma(x)(1-sigma(x))
    # allowing to recycle the hidden_layer_outputs computation
    hidden_error = hidden_layer_outputs[:, 1:] * (1 - hidden_layer_outputs[:, 1:]) * np.dot(output_error, output_weights.T[:, 1:])

    # partial derivatives
    hidden_pd = input_layer_outputs[:, :, np.newaxis] * hidden_error[: , np.newaxis, :]
    output_pd = hidden_layer_outputs[:, :, np.newaxis] * output_error[:, np.newaxis, :]

    # average for total gradients
    total_hidden_gradient = np.average(hidden_pd, axis=0)
    total_output_gradient = np.average(output_pd, axis=0)

    # update weights
    hidden_weights += - alpha * total_hidden_gradient
    output_weights += - alpha * total_output_gradient

# print the final outputs of the neural network on the inputs X
print("Output After Training: \n{}".format(output_layer_outputs))

Output After Training: 
[[0.01915115]
 [0.97462314]
 [0.97487913]
 [0.03401828]]


## Observations

- The sigmoid function has the computationally attractive property that its derivative does not require to recompute the fraction involving the exponent, but simply $\sigma'(x)=\sigma(x)(1-\sigma(x))$.

# Literature

Blum, A., & Rivest, R. L. (1989). Training a 3-node neural network is NP-complete. In Advances in neural information processing systems (pp. 494-501).

Rumelhart, D. E., Hinton, G. E., & Williams, R. J. (1985). Learning internal representations by error propagation (No. ICS-8506). California Univ San Diego La Jolla Inst for Cognitive Science.
